## CSC 177-02 Data Warehousing and Data Mining
### Mini-Project 1: Clustering
### 2016 US presedential election Twitter analysis

#### Group members: Aaron Enberg,

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [2]:
column_names = ['Name', 'screen_Name', 'User_ID', 
                'Followers_Count', 'Friends_Count', 
                'Location', 'Description', 'Created_At', 
                'Status_ID', 'Language', 'Place', 
                'Retweet_Count', 'Favorite_Count', 'Text']
tweets = pd.read_table('data/clinton_trump_tweets.txt', names=column_names, encoding='ISO-8859-1')
tweets.columns = tweets.columns.str.lower()

In [3]:
tweets.head()

,name,screen_name,user_id,followers_count,friends_count,location,description,created_at,status_id,language,place,retweet_count,favorite_count,text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,Cookie,Cookiemuffen,109945090,2154,2034,The American South,Got married after college. I don't regret starting a family instead of grad school. Proud Deplorable,Wed Oct 26 18:44:08 EEST 2016,791304413923213312,en,NaN,1937,0,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,Mark Hager,marksnark,167177185,204,448,Pittsburgh,"Hip, trendy, smart, funny, fit, lobbyist. U? Boilerplate: these thoughts are my own, not anyone else's. Hmmmkay?",Wed Oct 26 00:33:20 EEST 2016,791029904733331457,en,NaN,891,0,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,Capitalist Creations,aaronjhoddinott,1191022351,775,154,Canada,"Entrepreneur, startup investor, political junkie, free market supporter, beer connoisseur, dad and dog lover. Also a golf enthusiast despite my lack of skill.",Fri Oct 28 05:05:10 EEST 2016,791823089700962304,en,NaN,7,0,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.


In [4]:
tweets.shape

(5250980, 14)

In [5]:
tweets.dtypes

name               object
screen_name        object
user_id            int64 
followers_count    int64 
friends_count      int64 
location           object
description        object
created_at         object
status_id          int64 
language           object
place              object
retweet_count      int64 
favorite_count     int64 
text               object
dtype: object

## Preprocessing

In [6]:
pattern = r'^RT\s'
 
# matches retweets and removes them
tweets = tweets[tweets.text.str.match(pattern) == False]


In [7]:
tweets.head(n = 15)

,name,screen_name,user_id,followers_count,friends_count,location,description,created_at,status_id,language,place,retweet_count,favorite_count,text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,robert2266,robert2266,17101060,845,938,The Universe,The Dark Lord,Fri Oct 28 14:41:06 EEST 2016,791968028191711237,en,NaN,0,0,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,neddyo,neddyo,16818809,1400,379,Long Island and beyond...,"You should be digging it while it's happening - Zappa; I don't know how you do it, man. - Anastasio #minimix #recommNeds #NYCSOTW",Mon Oct 31 08:06:52 EET 2016,792971077836124160,en,NaN,0,1,Hulk smash!
12,Alexandria Taylor,twentythreeofme,163712593,1342,1346,"Portland, Oregon",Activist | Creator | Educator | #FemTube Host | Passion Project: #DisabilitiesOnYT Panels | Want to participate? https://t.co/IYy6JVqmo5,Tue Oct 25 19:41:52 EEST 2016,790956555030061056,en,"PlaceJSONImpl{name='Washington', streetAddress='null', countryCode='US', id='bc3a38d3d5999b4b', country='United States', placeType='admin', url='https://api.twitter.com/1.1/geo/id/bc3a38d3d5999b4b.json', fullName='Washington, USA', boundingBoxType='Polygon', boundingBoxCoordinates=[[Ltwitter4j.GeoLocation;@5ad5215f], geometryType='null', geometryCoordinates=null, containedWithIn=[]}",0,7,"Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"
13,Harry Binford,CaptainNormal,2447279666,895,1224,USA,"Proud Christian husband, frequent shooting range attendee, and former law enforcement. Lifetime member of the NRA. Pro-life, Jesus is Lord. America first.",Thu Oct 27 21:44:44 EEST 2016,791712249685676032,en,NaN,0,0,@WayneDupreeShow #climatechange is the democrats' attempt to play God
18,Matthew Carl Barnard,matthewcbarnard,231921777,350,668,"Milwaukee, Wisconsin",Trying to put one foot forward each day towards living my dream. I have thoughts along the way. Enjoy.,Wed Oct 26 04:23:47 EEST 2016,791087900368789504,en,NaN,0,0,OH MY GOD. TEARS. https://t.co/8odjnHZwRK
20,mystikal knight,_Poochivicious,235657662,777,3236,On An Ultra Light Beams,my life is currently a master p sneaker,Fri Oct 28 15:04:44 EEST 2016,791973973106057216,en,NaN,0,0,"no lie, I been waiting for tony's story https://t.co/S5i4ESNilb"
21,Jim Cook,valukind,201499452,1413,620,New York City,"Internet marketer, college grad, Air Force Vet.Capt.,like people, cats, football,fine dining and helping others to achieve.",Fri Oct 28 03:22:20 EEST 2016,791797209335418880,en,NaN,0,0,How a dot-com millionaire became a social media crusader against solar amendment: It was a violation of Googl... https://t.co/ychypPZpKd


In [8]:
# match all hashtags and mentions in a tweet, ignoring possible email addresses
pattern = r'(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z_]+[A-Za-z0-9_]+)|(?<=^|(?<=[^a-zA-Z0-9-\.]))#([A-Za-z_]+[A-Za-z0-9_]+)'

""" extractall() returns a DataFrame with a MultiIndex:
    First index is our original index. Second index is "match" which is a running
    total of the number of occurences of hashtags and mentions for a particular 
    tweet. So, a match = 0 does NOT mean there are no matches, actually it's the 
    first occurence of a hashtag or mention found in the tweet (index starts from 0)  """
mention_hashtag = tweets.text.str.extractall(pattern)

In [9]:
mention_hashtag.columns = ['mentions', 'hashtags']
mention_hashtag.head()

mentions hashtags
  match                       
0 0      NWAJimmy      NaN    
2 0      gaystoner821  NaN    
6 0      NaN           infosec
  1      NaN           Intel  
  2      NaN           ACM

In [10]:
''' Turn the MultiIndex Dataframe into a regular single index with 
    same index from training data. This is now our basket of tweets
    containing atleast one mention or hastag '''
mention_hashtag = mention_hashtag.reset_index().set_index('level_0')
del mention_hashtag.index.name
mention_hashtag.drop(['match'], axis=1, inplace=True)

In [11]:
mention_hashtag

,mentions,hashtags
0,NWAJimmy,NaN
2,gaystoner821,NaN
6,NaN,infosec
6,NaN,Intel
6,NaN,ACM
6,NaN,IEEE
12,Haylie_Bre,NaN
13,WayneDupreeShow,NaN
13,NaN,climatechange
22,tansleyemiley69,NaN


In [12]:
mentions_hashtags = pd.concat([mention_hashtag.mentions, mention_hashtag.hashtags]).dropna()
mentions_hashtags = mentions_hashtags.to_frame(name='mentions_hashtags')

In [19]:
tweets = tweets.join(mentions_hashtags, how='inner').head(n=30)

In [20]:
tweets
# tweets.groupby(['screen_name', 'mentions']).sum()

,name,screen_name,user_id,followers_count,friends_count,location,description,created_at,status_id,language,place,retweet_count,favorite_count,text,mentions_hashtags
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!,NWAJimmy
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.,gaystoner821
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv,infosec
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv,Intel
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv,ACM
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv,IEEE
12,Alexandria Taylor,twentythreeofme,163712593,1342,1346,"Portland, Oregon",Activist | Creator | Educator | #FemTube Host | Passion Project: #DisabilitiesOnYT Panels | Want to participate? https://t.co/IYy6JVqmo5,Tue Oct 25 19:41:52 EEST 2016,790956555030061056,en,"PlaceJSONImpl{name='Washington', streetAddress='null', countryCode='US', id='bc3a38d3d5999b4b', country='United States', placeType='admin', url='https://api.twitter.com/1.1/geo/id/bc3a38d3d5999b4b.json', fullName='Washington, USA', boundingBoxType='Polygon', boundingBoxCoordinates=[[Ltwitter4j.GeoLocation;@5ad5215f], geometryType='null', geometryCoordinates=null, containedWithIn=[]}",0,7,"Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv",Haylie_Bre
13,Harry Binford,CaptainNormal,2447279666,895,1224,USA,"Proud Christian husband, frequent shooting range attendee, and former law enforcement. Lifetime member of the NRA. Pro-life, Jesus is Lord. America first.",Thu Oct 27 21:44:44 EEST 2016,791712249685676032,en,NaN,0,0,@WayneDupreeShow #climatechange is the democrats' attempt to play God,WayneDupreeShow
13,Harry Binford,CaptainNormal,2447279666,895,1224,USA,"Proud Christian husband, frequent shooting range attendee, and former law enforcement. Lifetime member of the NRA. Pro-life, Jesus is Lord. America first.",Thu Oct 27 21:44:44 EEST 2016,791712249685676032,en,NaN,0,0,@WayneDupreeShow #climatechange is the democrats' attempt to play God,climatechange
22,McDonald's,McDonaldsCorp,111679943,143939,11621,"Oak Brook, IL","Learn about McDonalds history, people, food, & restaurants around the world

In [39]:
tweets = tweets.groupby('screen_name')['mentions_hashtags'].value_counts().to_frame(name='frequency')

In [45]:
tweets.pivot_table(values='frequency', index=['screen_name'], columns='item')

frequency
screen_name     mentions_hashtags           
45CalWrench     HillNO             1        
Andrea_Rosu     MutinyWrestling    1        
BIGGMARV        Timberland         1        
Bryan_Vehonsky  KyleOrl            1        
                ZekeJMiller        1        
CaptainNormal   WayneDupreeShow    1        
                climatechange      1        
Cebel6          NWAJimmy           1        
McDonaldsCorp   tansleyemiley69    1        
ShieldGirl127   HIAC               1        
                RawWomensChampion  1        
UnBitterEnd2013 SaraMitchellTV     1        
chrisf91        mlombardifoxtv     1        
                ochocinco          1        
davidjwalling   ACM                1        
                IEEE               1        
                Intel              1        
                infosec            1        
minasmith64     MegynKelly         1        
                Trump2016          1        
nolaguy_phd     gaystoner821       1        
thatoaklandkid  ESPNNBA            1        
                KDTrey5            1        
                MamaDurant         1        
                NBA                1        
                NBAonTNT           1        
                Oakland            1        
                espn               1        
                warriors           1        
twentythreeofme Haylie_Bre         1

In [15]:
# count up all the occurences for each distinct mention
count_mentions = mention_hashtag.mentions.value_counts()
# turn the series back into a dataframe 
count_mentions = count_mentions.reset_index(name="count")
# retrieve only those mentions having atleast 20 occurences 
mentions_20 = count_mentions.query("count >= 19")

In [16]:
# do the same thing for the hashtags
count_hashtags = mention_hashtag.hashtags.value_counts()
count_hashtags = count_hashtags.reset_index(name="count") 
count_hashtags.query("count >= 19")

,index,count
0,WorldSeries,9933
1,Trump,6688
2,quote,6274
3,MAGA,6075
4,news,6070
5,NowPlaying,4895
6,PJNET,4175
7,Hillary,3946
8,ImWithHer,3930
9,RallyTogether,3516


In [17]:
''' join back the rest of the data for all 
    users with atleast 20 hashtags + mentions '''
#users_20 = pd.concat([tweets, mention_hashtag_20], axis=1, join_axes=[mention_hashtag_20.index])

' join back the rest of the data for all \n    users with atleast 20 hashtags + mentions '

In [18]:
# number of times a user has used a hastag 
user_hashtag_frequency = users_20.groupby(['screen_name', 'hashtags']).size().reset_index(name='hashtag_frequency')
#user_hashtag_frequency = user_hashtag_frequency.to_frame()
user_hashtag_frequency = user_hashtag_frequency.pivot_table('hashtag_frequency', 'screen_name', 'hashtags', fill_value=0)
user_hashtag_frequency.head()

NameError: name 'users_20' is not defined

In [ ]:
# number of times a user has mentioned another user
user_mention_frequency = users_20.groupby(['screen_name', 'mentions']).size().reset_index(name='mention_frequency')
user_mention_frequency = user_mention_frequency.pivot_table('mention_frequency', 'screen_name', 'mentions', fill_value=0)
user_mention_frequency.head()

In [ ]:
# somethings wrong here, text from index 12 which contains a mention is not listed here
#user_mention_hashtag_frequency = user_mention_frequency.merge(user_hashtag_frequency, how='outer')
#user_mention_hashtag_frequency
user_mention_hashtag_frequency = pd.concat([user_mention_frequency, user_hashtag_frequency], axis=1).fillna(value=0)

In [ ]:
user_mention_hashtag_frequency.head()


In [ ]:
user_mention_hashtag_frequency = user_mention_hashtag_frequency[user_mention_hashtag_frequency.columns[user_mention_hashtag_frequency.sum()>=20]]

In [ ]:
user_mention_hashtag_frequency.head()